In [ ]:
from datetime import datetime, timedelta
from collections import defaultdict
import calendar

def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    # Determine the start and end of the target month
    year, month = map(int, target_month.split("-"))
    month_start = datetime(year, month, 1)
    last_day = calendar.monthrange(year, month)[1]
    month_end = datetime(year, month, last_day)

    groups = defaultdict(lambda: {"qty": 0, "amount": 0.0})
    total_revenue = 0.0

    for item in item_list:
        # Convert and clean up input data
        start_date = datetime.strptime(item["start_date"], "%Y-%m-%d")
        stop_date = datetime.strptime(item["stop_date"], "%Y-%m-%d")
        rate = float(item["rate"])
        qty = int(item["qty"])

        # Determine if item is active in the target month
        active_start = max(start_date, month_start)
        active_end = min(stop_date, month_end)

        if active_start > active_end:
            continue  # Skip inactive items

        # Calculate active days and total days in month
        active_days = (active_end - active_start).days + 1
        total_days = (month_end - month_start).days + 1
        billing_period = f"{active_start.date()} to {active_end.date()}"

        # Calculate prorated amount
        prorated_amount = (rate * qty) * (active_days / total_days)

        # Group key: item_code, rate, billing_period
        group_key = (item["item_code"], rate, billing_period)
        groups[group_key]["qty"] += qty
        groups[group_key]["amount"] += prorated_amount

    # Construct final output
    line_items = []
    for (item_code, rate, billing_period), data in groups.items():
        line_items.append({
            "item_code": item_code,
            "rate": rate,
            "qty": data["qty"],
            "amount": round(data["amount"], 2),
            "billing_period": billing_period
        })
        total_revenue += data["amount"]

    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }
